### 머신러닝 MultinomialNB test

In [2]:
from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from konlpy.tag import Mecab, Okt, Kkma
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding
from gensim.models import Word2Vec, FastText
try:
    from koeda import AEDA, EDA, RD, RI, SR, RS
except ImportError:
    !pip install koeda
    from koeda import AEDA, EDA, RD, RI, SR, RS

In [3]:
def preprocess(data):
    def _aug_setup():
        global config

        if config['aug']['mode'] == 'e':
            augmenter = EDA(morpheme_analyzer=config['morp'], alpha_sr=config['aug']['sr']['a'], alpha_ri=config['aug']['ri']['a'], alpha_rs=config['aug']['rs']['a'], prob_rd=config['aug']['rd']['a'])
            p = (config['aug']['sr']['p'], config['aug']['ri']['p'], config['aug']['rs']['p'], config['aug']['rd']['p'])
        elif config['aug']['mode'] == 'a':
            augmenter = AEDA(morpheme_analyzer=config['morp'], punc_ratio=0.3)
            p = max(config['aug']['sr']['p'], config['aug']['ri']['p'], config['aug']['rs']['p'], config['aug']['rd']['p'])
        else:
            augmenter = []
            if config['aug']['rd']['p'] != 0.0:
                augmenter.append(
                    (RD(morpheme_analyzer=config['morp']), config['aug']['rd']['p'])
                )

            if config['aug']['ri']['p'] != 0.0:
                augmenter.append(
                    (RI(morpheme_analyzer=config['morp'], stopword=config['aug']['stopword']), config['aug']['ri']['p'])
                )

            if config['aug']['sr']['p'] != 0.0:
                augmenter.append(
                    (SR(morpheme_analyzer=config['morp'], stopword=config['aug']['stopword']), config['aug']['sr']['p'])
                )

            if config['aug']['rs']['p'] != 0.0:
                augmenter.append(
                    (RS(morpheme_analyzer=config['morp']), config['aug']['rs']['p'])
                )
            p = None

        return augmenter, p

    def _aug(text):
        global config
        nonlocal augmenter
        nonlocal p

        if isinstance(augmenter, list):
            result = text

            for aug, p in augmenter:
                result = aug(result, p, config['aug']['repetition'])
        else:
            result = augmenter(text, p, config['aug']['repetition'])

        return result

    global config

    if config['is_cut']:
        # 'conversation' 열의 각 항목에 대한 문자 수를 계산합니다.
        data['conversation_length'] = data['conversation'].apply(len)

        # 문자 수가 400 미만인 행만 선택합니다.
        data = data[data['conversation_length'] < config['cut_point']]

    if config['is_aug']:
        # 중복 augmenter 생성 방지를 위해서 처음 한번에 생성
        augmenter, p = _aug_setup()
        
        # 랜덤하게 행 선택 (예: 전체 행의 20%를 선택)
        random_indices = np.random.choice(data.index, size=int(len(data) * config['aug']['ratio']), replace=False)

        # 선택된 행에 대해 Random swap 함수 적용
        augmented_rows = data.loc[random_indices, 'conversation'].apply(_aug)

        # 증강된 데이터를 복사하고, 'text' 열에 증강된 텍스트를 삽입
        new_rows = data.loc[random_indices].copy()
        new_rows['conversation'] = augmented_rows
        
        # 형태소 분석 실행
        data['conversation'] = data['conversation'].apply(config['morp'].morphs)

        # 원본 데이터프레임에 증강된 데이터 추가
        data = pd.concat([data, new_rows])
    else:
        data['conversation'] = data['conversation'].apply(config['morp'].morphs)

    if config['is_stopword']:
        pass

    return data

In [14]:
config = {
    'is_preprocess': True, # 전처리 여부
    'is_dnn': True, # 딥러닝 사용 여부
    'morp': Kkma(), # 형태소 분석기 변경,
    'is_aug': False, # 데이터 augment 사용 여부
    # 데이터 augmentation 파라미터의 경우 아래의 글들을 참고하면 좋음
    # https://yeon22.tistory.com/203
    # https://catsirup.github.io/ai/2020/04/21/nlp_data_argumentation.html
    'aug': {
        'ratio': 0.3, # 적용할 데이터의 비율
        # 사용하기 싫은 것은 p의 값을 0.0으로 지정 한다.
        'rd': {
            # 여기서 a는 사실 확률이다.
            # API 통일을 위해 a로 표기했지만, prob_rd가 본명칭
            'a': 0.3,
            'p': 0.4,
        }, # RandomDeletion
        'ri': {
            'a': 0.3, # alpha 값이고 데이터 증강 기법의 강도를 의미한다.
            'p': 0.4, # p는 증강 기법이 "얼마나 자주" 적용될지
        }, # RandomInsertion
        'sr': {
            'a': 0.3,
            'p': 0.4,
        }, # SynonymReplacement
        'rs': {
            'a': 0.3,
            'p': 0.4,
        }, # RandomSwap
        'mode': 'e', # EDA: e, AEDA: a, other: o
        'stopword': True,
        'repetition': 1 # 반복 여부인 것 같아요.
    },
    'is_cut': True, # 단어길이 자를건지
    'cut_point': 400, # 자르는 기준
    'is_word2vec': False, # word2vec 사용 여부
    'is_fasttext': True, # fasttext 사용 여부
    'is_stopword': False # syh님
}

In [15]:
print('start')
# 데이터는 그대로고 모델만 수정해서 확인할 경우
# 중복으로 읽고 전처리하는 대신 FIXED 변수를 통해 제어하세요.
# 'df' in globals() 은 변수의 존재 여부를 판단합니다.
FIXED = True
if not FIXED and 'df' in not globals():
    print('get data')
    df = pd.read_csv('train.csv')

    if config['is_preprocess']:
        print('preprocess')
        df = preprocess(df)
else:
    print('data was fixed')

print('start train')
if config['is_dnn']:
    print('select dnn')
    # 가정: 입력 크기는 1000, 출력 클래스는 2
    max_words = 1000
    output_dim = 4

    print('tokenize')
    # Tokenizer를 생성하고 텍스트 데이터에 적합시킵니다.
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(df['conversation'])

    # 텍스트를 정수 인덱스 시퀀스로 변환합니다.
    sequences = tokenizer.texts_to_sequences(df['conversation'])

    # 시퀀스의 길이를 맞추기 위해 패딩을 추가합니다.
    data_pad = pad_sequences(sequences, padding='post')

    # class 열을 숫자로 변환
    encoder = LabelEncoder()
    df['class'] = encoder.fit_transform(df['class'])

    print('split dataset')
    x_train, x_test, y_train, y_test = train_test_split(data_pad, df['class'], test_size=0.2)

    # word2vec
    assert not (config['is_word2vec'] and config['is_fasttext']), 'you have to choose only one among word2vec and fasttext'
    if config['is_word2vec'] or config['is_fasttext']:
        word_index = tokenizer.word_index
        index_word = {idx: word for word, idx in word_index.items()}
        recovered = [[index_word[idx] for idx in seq if idx != 0] for seq in sequences]
        
        if config['is_word2vec']:
            print('select word2vec')
            # Word2Vec 모델 학습
            word2vec_model = Word2Vec(sentences=recovered, 
                                    size=100, 
                                    window=5, 
                                    min_count=1, 
                                    workers=4,
                                    sg=0)

            # 단어 인덱스와 임베딩 매트릭스 생성
            vocab_size = len(word2vec_model.wv.vocab) + 1  # +1 for padding
            embedding_dim = word2vec_model.wv.vector_size
            embedding_matrix = np.zeros((vocab_size, embedding_dim))

            for i, word in enumerate(word2vec_model.wv.vocab):
                embedding_matrix[i] = word2vec_model.wv[word]

            # Embedding layer with pre-trained Word2Vec weights
            embedding_layer = Embedding(vocab_size, 
                                        embedding_dim, 
                                        weights=[embedding_matrix], 
                                        input_length=244,  # 뭐 차원이 다르다 이러면 요거 건드려 보세요.
                                        trainable=False)  # Keep embeddings fixed
        elif config['is_fasttext']:
            print('select fasttext')
            fasttext_model = FastText(
                sentences=recovered,
                window=5,
                min_count=5,
                workers=4,
                sg=1
            )

            embedding_dim = fasttext_model.vector_size
            vocab_size = len(fasttext_model.wv.vocab)
            embedding_matrix = np.zeros((vocab_size, embedding_dim))

            for i, word in enumerate(fasttext_model.wv.vocab):
                embedding_matrix[i] = fasttext_model.wv[word]

            embedding_layer = Embedding(vocab_size,
                                        embedding_dim,
                                        weights=[embedding_matrix],
                                        input_length=244,
                                        trainable=False)

    else:
        print('select normal')
        embedding_layer = Embedding(len(tokenizer.word_index)+1, 128, input_length=data_pad.shape[1])

    print('create model')
    # 모델을 생성합니다.
    model = Sequential()
    model.add(embedding_layer)
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(len(df['class'].unique()), activation='softmax'))

    # 모델을 컴파일합니다.
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    print('model fit')
    # 모델을 학습합니다.
    model.fit(x_train, y_train, epochs=10)

    print('model evaluate')
    model.evaluate(x=x_test, y=y_test)
else:
    print('select ml')
    vectorizer = CountVectorizer()

    x_train = vectorizer.fit_transform(df['conversation'])
    y_train = df['class']

    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2)


    model = MultinomialNB()
    model.fit(x_train, y_train)

    score = model.score(x_test, y_test)
    print('Accuracy:', score)

start
data was fixed
start train
select dnn
tokenize
split dataset
select fasttext
create model
model fit
Epoch 1/10
 5/93 [>.............................] - ETA: 1s - loss: 2.8289 - accuracy: 0.3250 

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_3/embedding_3/embedding_lookup' defined at (most recent call last):
    File "c:\Users\tama0\anaconda3\envs\dlton210\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\tama0\anaconda3\envs\dlton210\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\tama0\anaconda3\envs\dlton210\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\tama0\anaconda3\envs\dlton210\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\Users\tama0\anaconda3\envs\dlton210\lib\site-packages\ipykernel\kernelapp.py", line 728, in start
      self.io_loop.start()
    File "c:\Users\tama0\anaconda3\envs\dlton210\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "

필요시 참고 링크
KoEDA
https://github.com/toriving/KoEDA